<a href="https://colab.research.google.com/github/Nirzaree/Detecting-Roads-from-Satellite-Images/blob/master/Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import os 
import cv2
import glob

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#!pip install git+https://github.com/qubvel/segmentation_models

### Load dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 64

In [ ]:
data_dir = '/content/drive/MyDrive/Data/MasachusettsRoads/'

In [ ]:
!find $data_dir -type d -print

/content/drive/MyDrive/Data/MasachusettsRoads/
/content/drive/MyDrive/Data/MasachusettsRoads/road_segmentation_ideal
/content/drive/MyDrive/Data/MasachusettsRoads/road_segmentation_ideal/training
/content/drive/MyDrive/Data/MasachusettsRoads/road_segmentation_ideal/training/input
/content/drive/MyDrive/Data/MasachusettsRoads/road_segmentation_ideal/training/output


In [ ]:
base_dir = os.path.join(os.path.dirname(data_dir))
train_dir = os.path.join(base_dir,'road_segmentation_ideal/training/input/')

train_mask_dir = os.path.join(base_dir,'road_segmentation_ideal/training/output/')


## Understand data

In [ ]:
num_training_img = len(os.listdir(train_dir))
# num_test_img = len(os.listdir(test_dir))

num_training_masks = len(os.listdir(train_mask_dir))
# num_test_masks = len(os.listdir(test_mask_dir))

print("Total training images",num_training_img)
# print("Total test images",num_test_img)


print("Total training masks",num_training_masks)
# print("Total test masks",num_test_masks)

Total training images 69
Total training masks 59


In [ ]:
os.listdir('/content/drive/MyDrive/Data/MasachusettsRoads/road_segmentation_ideal/')

['training']

## Data Preparation

In [ ]:
img_names = glob.glob(train_dir + '/*.png')

In [ ]:
img_names.sort()
# print(img_names)

In [ ]:
mask_names = glob.glob(train_mask_dir + '/*.png')

In [ ]:
mask_names.sort()

In [ ]:
img_names = [x for x in img_names if os.path.basename(x) in [os.path.basename(x) for x in mask_names]]

In [ ]:
mask_names = [x for x in mask_names if os.path.basename(x) in [os.path.basename(x) for x in img_names]]

In [ ]:
print("Final training images",len(img_names))
# print("Total test images",num_test_img)


print("Final training masks",len(mask_names))

Final training images 59
Final training masks 59


### Plot few images and masks 

In [ ]:
# plt.figure(figsize = (20,10))
# for ind in range(4):
#     plt.subplot(1,4,ind+1)
#     sample_img = cv2.imread(img_names[ind])
#     plt.imshow(sample_img)
    
# plt.figure(figsize = (20,10))
# for ind in range(4):
#     plt.subplot(1,4,ind+1)
#     sample_mask = cv2.imread(mask_names[ind])
#     plt.imshow(sample_mask)

## Read data into numpy arrays

In [ ]:
train_images = []
train_masks = []
for ind in range(30):
    img = cv2.imread(img_names[ind])
    mask = cv2.imread(mask_names[ind])
    train_images.append(img)
    train_masks.append(mask)

In [ ]:
train_images = np.array(train_images)

In [ ]:
train_masks = np.array(train_masks)

### Normalize data

In [ ]:
train_images = train_images/255.0

In [ ]:
np.unique(train_masks)

array([  0, 255], dtype=uint8)

In [ ]:
train_masks = train_masks/255.0

### Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train_images,
    train_masks,
    test_size = 0.2,
    random_state = 0
)

In [ ]:
print("Train Set")
print("Shape of images:",X_train.shape)
print("Shape of masks:",y_train.shape)

Train Set
Shape of images: (24, 1500, 1500, 3)
Shape of masks: (24, 1500, 1500, 3)


In [ ]:
print("Test Set")
print("Shape of images:",X_test.shape)
print("Shape of masks:",y_test.shape)


Test Set
Shape of images: (6, 1500, 1500, 3)
Shape of masks: (6, 1500, 1500, 3)


## Build Model

In [ ]:
import segmentation_models as sm
from segmentation_models import Unet
from tensorflow.keras.optimizers import Adam
from keras.layers import Input
from segmentation_models.losses import bce_jaccard_loss, bce_dice_loss
from segmentation_models.metrics import iou_score

In [ ]:
sm.set_framework('tf.keras')

sm.framework() #helper: https://stackoverflow.com/questions/67792138/attributeerror-module-keras-utils-has-no-attribute-get-file-using-segmentat

'tf.keras'

In [ ]:
model = Unet(backbone_name = 'efficientnetb0', 
             encoder_weights='imagenet', 
             encoder_freeze = False)

model.compile(optimizer = Adam(),
              loss=bce_jaccard_loss,
              metrics=[iou_score])

In [ ]:
IMAGE_HEIGHT = 1500
IMAGE_WIDTH = 1500

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 stem_conv (Conv2D)             (None, None, None,   864         ['input_6[0][0]']                
                                32)                                                               
                                                                                                  
 stem_bn (BatchNormalization)   (None, None, None,   128         ['stem_conv[0][0]']              
                                32)                                                         

### Model callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

# reduces learning rate on plateau
lr_reducer = ReduceLROnPlateau(factor=0.1,
                               cooldown= 10,
                               patience=10,verbose =1,
                               min_lr=0.1e-5)

mode_autosave = ModelCheckpoint("./weights/road.efficientnetb0imgsize.h5",
                                monitor='val_iou_score',
                                mode = 'max',
                                save_best_only=True, 
                                verbose=1, 
                                period =10)

# stop learining as metric on validatopn stop increasing
early_stopping = EarlyStopping(patience=10, verbose=1, mode = 'auto') 

# tensorboard for monitoring logs
tensorboard = TensorBoard(log_dir='./logs/tenboard', histogram_freq=0,
                          write_graph=True, write_images=False)

callbacks = [mode_autosave, lr_reducer, tensorboard, early_stopping]

In [ ]:
history = model.fit(X_train,
                    y_train, 
                    validation_data = (X_test,y_test),                 
                    epochs = 30,
                    batch_size = 15,
                    verbose = 1,
                    callbacks = callbacks)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/30


ValueError: ignored

In [ ]:
X_train.shape

(24, 1500, 1500, 3)

In [ ]:
X_test.shape

(6, 1500, 1500, 3)

In [ ]:
y_train.shape

(24, 1500, 1500, 3)